In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src directory to path for imports
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from design import generate_reentrant_cell, calculate_poisson_ratio, estimate_capacitance_change
from utils import distance, polygon_area
from test_setup import MockLCRMeter, read_capacitance

print("✓ All modules imported successfully")
print("✓ Ready for auxetic sensor simulation")


In [ ]:
# Parameter sweep: effect of re-entrance angle on Poisson's ratio
angles = np.linspace(5, 85, 50)  # degrees
poisson_ratios = [calculate_poisson_ratio(1.0, alpha) for alpha in angles]

plt.figure(figsize=(10, 6))
plt.plot(angles, poisson_ratios, 'b-', linewidth=2, label='Theoretical')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5, label='Conventional limit')
plt.xlabel('Re-entrance Angle α (degrees)')
plt.ylabel("Poisson's Ratio ν")
plt.title('Auxetic Behavior vs Re-entrance Angle')
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(-2, 0.5)

# Highlight optimal range
optimal_range = (angles >= 20) & (angles <= 60)
plt.fill_between(angles[optimal_range], -2, 0.5, alpha=0.2, color='green', 
                 label='Practical range')
plt.legend()
plt.show()

print(f"At α=30°: ν = {calculate_poisson_ratio(1.0, 30.0):.3f}")
print(f"At α=45°: ν = {calculate_poisson_ratio(1.0, 45.0):.3f}")
print(f"At α=60°: ν = {calculate_poisson_ratio(1.0, 60.0):.3f}")


In [ ]:
def plot_cell(a, b, alpha, title=""):
    """Plot a single re-entrant cell."""
    cell = generate_reentrant_cell(a, b, alpha)
    nodes = np.array(cell["nodes"])
    edges = cell["edges"]
    
    plt.figure(figsize=(8, 8))
    
    # Plot edges
    for edge in edges:
        start, end = edge
        x_coords = [nodes[start][0], nodes[end][0]]
        y_coords = [nodes[start][1], nodes[end][1]]
        plt.plot(x_coords, y_coords, 'b-', linewidth=3)
    
    # Plot nodes
    plt.scatter(nodes[:, 0], nodes[:, 1], c='red', s=100, zorder=5)
    
    # Annotate nodes
    for i, (x, y) in enumerate(nodes):
        plt.annotate(f'{i}', (x, y), xytext=(5, 5), textcoords='offset points')
    
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title(f'Re-entrant Cell: a={a}m, b={b}m, α={alpha}° {title}')
    
    # Calculate and display area
    area = polygon_area(cell["nodes"])
    plt.text(0.02, 0.98, f'Area: {area:.4f} m²', transform=plt.gca().transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))
    
    plt.show()
    return cell

# Plot cells with different angles
cell_30 = plot_cell(1.0, 0.1, 30.0, "(Moderate auxetic)")
cell_45 = plot_cell(1.0, 0.1, 45.0, "(Strong auxetic)")
cell_60 = plot_cell(1.0, 0.1, 60.0, "(Very strong auxetic)")


In [ ]:
# Sensor parameters
initial_gap = 1e-3  # 1mm capacitor gap
strains = np.linspace(0, 0.05, 100)  # 0-5% strain

# Compare different auxetic designs
angles_to_compare = [30, 45, 60]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(12, 8))

for i, alpha in enumerate(angles_to_compare):
    nu = calculate_poisson_ratio(1.0, alpha)
    capacitance_changes = [estimate_capacitance_change(initial_gap, strain, nu) 
                          for strain in strains]
    
    plt.subplot(2, 2, 1)
    plt.plot(strains * 100, np.array(capacitance_changes) * 100, 
             color=colors[i], linewidth=2, label=f'α={alpha}° (ν={nu:.2f})')

plt.xlabel('Applied Strain (%)')
plt.ylabel('Capacitance Change (%)')
plt.title('Sensor Response: Auxetic vs Conventional')
plt.grid(True, alpha=0.3)
plt.legend()

# Add conventional material for comparison
nu_conventional = 0.3
cap_changes_conv = [estimate_capacitance_change(initial_gap, strain, nu_conventional) 
                   for strain in strains]
plt.plot(strains * 100, np.array(cap_changes_conv) * 100, 
         'k--', linewidth=2, label=f'Conventional (ν={nu_conventional})')
plt.legend()

# Sensitivity analysis
plt.subplot(2, 2, 2)
sensitivities = []
for alpha in angles:
    nu = calculate_poisson_ratio(1.0, alpha)
    # Sensitivity = dC/dε at small strain
    sensitivity = estimate_capacitance_change(initial_gap, 0.001, nu) / 0.001
    sensitivities.append(sensitivity * 100)  # Convert to %/%

plt.bar(range(len(angles)), sensitivities, color=colors)
plt.xlabel('Design')
plt.ylabel('Sensitivity (%/% strain)')
plt.title('Sensor Sensitivity Comparison')
plt.xticks(range(len(angles)), [f'α={a}°' for a in angles])
plt.grid(True, alpha=0.3)

# Mock measurement simulation
plt.subplot(2, 2, 3)
mock_lcr = MockLCRMeter(base_capacitance=10e-12)  # 10pF baseline

# Simulate measurement under varying load
time_points = np.linspace(0, 10, 100)
applied_strain = 0.02 * np.sin(2 * np.pi * 0.1 * time_points)  # Sinusoidal loading
nu_sensor = calculate_poisson_ratio(1.0, 45.0)  # Use 45° design

measured_caps = []
for strain in applied_strain:
    # Simulate capacitance change
    relative_change = estimate_capacitance_change(initial_gap, abs(strain), nu_sensor)
    new_cap = mock_lcr.base_capacitance * (1 + relative_change)
    mock_lcr.base_capacitance = new_cap  # Update for next measurement
    cap_reading = read_capacitance(mock_lcr)
    measured_caps.append(cap_reading * 1e12)  # Convert to pF

plt.plot(time_points, measured_caps, 'g-', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Measured Capacitance (pF)')
plt.title('Simulated Dynamic Response')
plt.grid(True, alpha=0.3)

# Performance metrics
plt.subplot(2, 2, 4)
performance_data = []
for alpha in angles_to_compare:
    nu = calculate_poisson_ratio(1.0, alpha)
    max_change = estimate_capacitance_change(initial_gap, 0.05, nu) * 100  # At 5% strain
    sensitivity = estimate_capacitance_change(initial_gap, 0.001, nu) / 0.001 * 100
    performance_data.append([abs(max_change), abs(sensitivity)])

performance_data = np.array(performance_data)
x_pos = np.arange(len(angles_to_compare))
width = 0.35

plt.bar(x_pos - width/2, performance_data[:, 0], width, label='Max Change (%)', alpha=0.8)
plt.bar(x_pos + width/2, performance_data[:, 1], width, label='Sensitivity (%/%)', alpha=0.8)

plt.xlabel('Design Configuration')
plt.ylabel('Performance Metric')
plt.title('Performance Summary')
plt.xticks(x_pos, [f'α={a}°' for a in angles_to_compare])
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n=== Sensor Performance Summary ===")
for i, alpha in enumerate(angles_to_compare):
    nu = calculate_poisson_ratio(1.0, alpha)
    sensitivity = estimate_capacitance_change(initial_gap, 0.001, nu) / 0.001 * 100
    max_change = estimate_capacitance_change(initial_gap, 0.05, nu) * 100
    print(f"α={alpha}°: ν={nu:.3f}, Sensitivity={sensitivity:.2f}%/%, Max Change={max_change:.2f}%")
